In [68]:
from tkinter import *
from tkinter import ttk
import pandas as pd
import config

root = Tk()
root.title('Cash CRM')
root.iconbitmap()
root.geometry("1000x500")

#add style
style = ttk.Style()

#pick a theme
style.theme_use('default')

#configure colours
style.configure("Treeview",
        background="#D3D3D3",
        foreground="black",
        rowheight=25,
        fieldbackground="#D3D3D3")

#Change selected colour
style.map('Treeview',
    background=[('selected', "#347083")])

#Create a treeview Frame
tree_frame = Frame(root)
tree_frame.pack(pady=10)

#create a treeview scrollbar
tree_scroll = Scrollbar(tree_frame)
tree_scroll.pack(side=RIGHT, fill=Y)

#Create the treeview
my_tree = ttk.Treeview(tree_frame, yscrollcommand=tree_scroll.set, selectmode="extended")
my_tree.pack()

#configure scrollbar
tree_scroll.config(command=my_tree.yview)



In [69]:

from sqlalchemy import create_engine
from sqlalchemy import text
engine = create_engine(config.mysql_url + config.user + ":" + config.password + "@" + config.host_name + "/" + config.schema)


animals = pd.read_sql('SELECT * FROM simple_crm.animals', con=engine)
active_table = animals

In [70]:
my_tree['columns'] = list(active_table.columns)

#format columns
my_tree.column("#0", width=0, stretch=NO)
for column in my_tree['columns']:
    my_tree.column(column, anchor=CENTER, width=140)

#name headers
my_tree.heading("#0", text="", anchor=W)
for column in my_tree['columns']:
    my_tree.heading(column, text=column, anchor=CENTER)

In [71]:
#create striped row tags
my_tree.tag_configure('oddrow', background="white")
my_tree.tag_configure("evenrow", background="lightblue")

{}

In [72]:
#add our data to the screen
global count
count = 0

column_count = active_table.shape[1]

for record in active_table.values:
    if count % 2 == 0:
        my_tree.insert(parent='', index='end', iid=count, text='', values=(list(record[0:column_count])), tags=('evenrow',))
    else:
        my_tree.insert(parent='', index='end', iid=count, text='', values=(list(record[0:column_count])), tags=('oddrow',))
    #increment counter
    count += 1

In [73]:
#add buttons and record entry boxes
button_frame = LabelFrame(root, text="Commands")
button_frame.pack(fill='x', expand='yes', padx=20)

data_frame = LabelFrame(root, text="Record")
data_frame.pack(fill="x", expand='yes', padx=20)

In [74]:
#building boxes dynamically

bxs = []
for b in my_tree['columns']:
    b=Entry(data_frame)
    #i.pack()
    bxs.append(b)

lbls = []  
for l in my_tree['columns']:
    l=Label(data_frame, text=l)
    #i.pack()
    lbls.append(l)


def selectRecord(tree, labels,boxes):

    selected = tree.focus()

    values = tree.item(selected, 'values')

    n = 0

    for label in labels:
        label.grid(row=n, column=1, padx=10, pady=10)
        n += 1
        
    n = 0

    for box in boxes:
        box.grid(row=n, column=2, padx=10, pady=10)
        box.delete(0,END)
        box.insert(0,values[n])
        n += 1
        
    
    
def onDouble(selected):
    selectRecord(my_tree,lbls,bxs)

updates = []

def getRecord(boxes):
    for item in boxes:
        v = item.get()
        updates.append(v)
        print(v)

my_tree.bind("<Double-1>", onDouble)

'140172472593728onDouble'

In [75]:
#adding buttons

select_record_button = Button(button_frame, text="Select Record", command= lambda: selectRecord(my_tree,lbls,bxs))
select_record_button.grid(row=0, column=1, padx=10, pady=10)

select_record_button = Button(button_frame, text="Update Record", command= lambda: getRecord(bxs))
select_record_button.grid(row=0, column=2, padx=10, pady=10)

In [76]:
root.mainloop()